# Epistatic Analysis Notebook | ShiLab
---

## This is a Jupyter Notebook based on R kernel.

In [ ]:
# load library
library('EBEN')
library('r2d3')

In [ ]:
# load data
workspace <- './'
x_filename <- 'Geno.txt'
y_filename <- 'Pheno.txt'

x <- read.table(
  file = file.path(workspace, x_filename),
  header = TRUE,
  check.names=FALSE,
  row.names = 1
)
sprintf('x size: (%d, %d)', nrow(x), ncol(x))
x <- as.matrix(x)

y <- read.table(
  file = file.path(workspace, y_filename),
  header = TRUE,
  row.names = 1
)
sprintf('y size: (%d, %d)', nrow(y), ncol(y))
y <- as.matrix(y)

In [ ]:
nFolds <- 5
max_percentages_miss_val <- 0.2
seed <- 28213
set.seed(seed)

In [ ]:
# Filter the miRNA data with more than 20% missing data
x_filtered <- NULL
x_filtered_colnames <- NULL
criteria <- trunc(nrow(x) * (1 - max_percentages_miss_val))
for (i in 1:ncol(x)) {
  if (sum(as.numeric(x[, i]) != 0) > criteria) {
    x_filtered <- cbind(x_filtered, x[, i])
    x_filtered_colnames<-c(x_filtered_colnames, colnames(x)[i])
  }
}
colnames(x_filtered)<-x_filtered_colnames
# colnames of x_filtered is same with x

# Quantile normalization
x_filtered_normed <- x_filtered
for (sl in 1:ncol(x_filtered_normed)) {
  mat = matrix(as.numeric(x_filtered_normed[, sl]), 1)
  mat = t(apply(mat, 1, rank, ties.method = "average"))
  mat = qnorm(mat / (nrow(x_filtered_normed) + 1))
  x_filtered_normed[, sl] = mat
}

x_preprocessed <- x_filtered_normed
rm(x_filtered, x_filtered_normed, sl, mat)

In [ ]:
# Main effect estimated using EBEN
CV = EBelasticNet.GaussianCV(x_preprocessed, y, nFolds = nFolds, Epis = "no")
Blup1 = EBelasticNet.Gaussian(
  x_preprocessed,
  y,
  lambda = CV$Lambda_optimal,
  alpha = CV$Alpha_optimal,
  Epis = "no",
  verbose = 0
)
Blup_main_sig = Blup1$weight[which(Blup1$weight[, 6] <= 0.05), ]

In [ ]:
# Substract the main effect
index_main <- Blup_main_sig[, 1]
effect_main <- Blup_main_sig[, 3]
y_new <- as.matrix(y) - x_preprocessed[, index_main] %*% (as.matrix(effect_main))

In [ ]:
# Epistatic effect estimated using EBEN. This step may need a long time.
CV_epis = EBelasticNet.GaussianCV(x_preprocessed, y_new, nFolds = nFolds, Epis = "yes")
Blup_epis = EBelasticNet.Gaussian(
  x_preprocessed,
  y_new,
  lambda =  CV_epis$Lambda_optimal,
  alpha = CV_epis$Alpha_optimal,
  Epis = "yes",
  verbose = 0
)
Blup_epis_sig = Blup_epis$weight[which(Blup_epis$weight[, 6] <= 0.05), ]

In [ ]:
# Final run
main_epi_sig_id = rbind(Blup_main_sig[, 1:2], Blup_epis_sig[, 1:2])

x_sig <- NULL
for (i in 1:nrow(main_epi_sig_id)) {
  if (main_epi_sig_id[i, 1] == main_epi_sig_id[i, 2]) {
    x_sig <- cbind(x_sig, x_preprocessed[, main_epi_sig_id[i, 1]])
  }
  if (main_epi_sig_id[i, 1] != main_epi_sig_id[i, 2]) {
    col <- x_preprocessed[, main_epi_sig_id[i, 1]] * x_preprocessed[, main_epi_sig_id[i, 2]]
    x_sig <- cbind(x_sig, col)
  }
}

# Quantile normalization 
x_sig_qnormed <- x_sig
for (sl in 1:ncol(x_sig_qnormed)) {
  mat = matrix(as.numeric(x_sig_qnormed[, sl]), 1)
  mat = t(apply(mat, 1, rank, ties.method = "average"))
  mat = qnorm(mat / (nrow(x_sig_qnormed) + 1))
  x_sig_qnormed[, sl] = mat
}
rm(x_sig, sl, mat)

CV_full = EBelasticNet.GaussianCV(x_sig_qnormed, y, nFolds = nFolds, Epis = "no")
Blup_full = EBelasticNet.Gaussian(
  x_sig_qnormed,
  y,
  lambda =  CV_full$Lambda_optimal,
  alpha = CV_full$Alpha_optimal,
  Epis = "no",
  verbose = 0
)
Blup_full_sig =  Blup_full$weight[which(Blup_full$weight[, 6] <= 0.05),]
Blup_full_sig[,1:2] <- main_epi_sig_id[Blup_full_sig[,1],1:2]

main_result <- NULL
epsi_result <- NULL
for (i in 1:nrow(Blup_full_sig)) {
  if (Blup_full_sig[i, 1] == Blup_full_sig[i, 2]) {
    main_result <- rbind(main_result, c(colnames(x_preprocessed)[Blup_full_sig[i, 1]],Blup_full_sig[i,3:6]))
  }
  if (Blup_full_sig[i, 1] != Blup_full_sig[i, 2]) {
    epsi_result <- rbind(epsi_result, c(colnames(x_preprocessed)[Blup_full_sig[i, 1]],colnames(x_preprocessed)[Blup_full_sig[i, 2]],Blup_full_sig[i,3:6]))
  }
}

In [ ]:
# show head of main results
colnames(main_result)<- c('feature','coefficent value','posterior variance','t-value','p-value')
head(main_result)

In [ ]:
# show head of epis results
colnames(epsi_result)<- c('feature1','feature2','coefficent value','posterior variance','t-value','p-value')
head(epsi_result)

In [ ]:
# generate json
# get all epsitasis nodes (unique)
epsi_nodes <- c(epsi_result[, 1], epsi_result[, 2])
epsi_nodes <- unique(epsi_nodes)
json_str<-NULL
for (i in 1:length(epsi_nodes)) {
  f1 <- epsi_nodes[i]
  f2 <- matrix(epsi_result[epsi_result[, 1] == f1, ],ncol=6)
  f2[,2]<-sprintf('"epis.%s"', f2[,2])
  element <- sprintf('{"name":"epis.%s","size":%d,"effects":[%s]}', f1, nrow(f2), paste(f2[,2], collapse=',' ))
  json_str <- c(json_str,element)
}
json_str<-sprintf('[%s]',paste(json_str, collapse=',' ))
# circle network
r2d3(data=json_str, script = "vis_CN.js", css = "vis_CN.css")

In [ ]:
# for adjacent matrix
epsi_nodes <- c(epsi_result[, 1], epsi_result[, 2])
epsi_nodes <- unique(epsi_nodes)
nodes_str<-NULL
for (i in 1:length(epsi_nodes)) {
  f1 <- epsi_nodes[i]
  element<- sprintf('{"name":"%s","group":"Epistatic effect","rank": %d}',f1, 1)
  nodes_str <- c(nodes_str,element)
}
nodes_str<-sprintf('"nodes":[%s]',paste(nodes_str, collapse=',' ))
link_str<-NULL
for(i in 1:nrow(epsi_result)){
  source_index<- match(epsi_result[i,1], epsi_nodes)
  target_index<- match(epsi_result[i,2], epsi_nodes)
  coff<-as.numeric(epsi_result[i,3])
  element<-sprintf('{"source":%d, "target":%d, "coff": %f}',source_index-1, target_index-1, coff)
  link_str <- c(link_str,element)
}

link_str<-sprintf('"links":[%s]',paste(link_str, collapse=',' ))
am_json<-sprintf('{%s,%s}',nodes_str,link_str)
write(am_json,'am.json')
r2d3( d3_version = 4, script = "vis_AM.js", css = "vis_AM.css", dependencies = "d3-scale-chromatic.js")